In [37]:
from pandas import DataFrame
from json import loads
from requests import get, post


In [35]:
def get_credentials()->dict[str, str]:
    with open("credentials.json") as cfile:
        credentials = loads(cfile.read())
    return credentials

In [61]:
sat_credentials = get_credentials()

In [90]:
prod_api_url = "https://api.satws.com"
test_api_url = "https://api.sandbox.satws.com"
path_type = "credetials"
prod_api_key = "3a5faf1c2155c2ded9ca82aafa1bfa49"
test_api_key = "4e42092fd7c079c612f2942e8d1d15ad"
id_client = "9852142f-6a9e-4313-8eb9-20110fe33101"

headers = {
    "X-API-Key": prod_api_url,
    #"Content-Type": "application/json"
}

data = sat_credentials["rolando"]


In [ ]:
{
    '@context': '/contexts/Entrypoint',
    '@id': '/',
    '@type': 'Entrypoint',
    'certificate': '/certificates',
    'credential': '/credentials',
    'event': '/events',
    'export': '/exports',
    'extraction': '/extractions',
    'invoice': '/invoices/issued',
    'invoiceBatchPayment': '/invoices/batch-payments',
    'invoiceCreditNote': '/invoices/credit-notes',
    'invoicePayment': '/invoices/payments',
    'link': '/links',
    'organization': '/organizations',
    'scheduler': '/schedulers',
    'schedulerRule': '/schedulers/rules',
    'taxRetention': '/tax-retentions',
    'taxStatus': '/tax-status',
    'taxpayer': '/taxpayers',
    'taxpayerGroup': '/taxpayer-groups',
    'user': '/users',
    'webServiceRequest': '/web-service-requests',
    'webhookEndpoint': '/webhook-endpoints',
    'webhookRequest': '/webhook-requests'
    }


In [95]:
http_request = {
    "url": f"https://api.satws.com/credentials/{id_client}",
    "headers": {
        "X-API-Key": prod_api_url,
        "Content-Type": "application/json"
    }
}

In [96]:
http_response = get(**http_request)
response = loads(http_response.text)
print(http_response.text)


{"message":"Forbidden"}


In [120]:
import requests

API_URLS = {
    "prod": "https://api.satws.com",
    "test": "https://api.sandbox.satws.com",
}
headers = {'X-API-Key': '3a5faf1c2155c2ded9ca82aafa1bfa49'}


def get_all_credentials(env: str = "prod") -> DataFrame:
    path_type = "credentials"
    response = requests.get(f"{API_URLS[env]}/{path_type}", headers=headers)
    parsed_response = loads(response.text)
    return DataFrame(parsed_response["hydra:member"])


def get_credentials(_id="credentials") -> DataFrame:
    path_type = "credentials"
    response = requests.get(f"{API_URLS}/{path_type}", headers=headers)
    print(response.text)
    parsed_response = loads(response.text)
    return DataFrame(parsed_response["hydra:member"])


In [118]:
get_all_credentials()

,@id,@type,id,type,rfc,status,metadata,createdAt,updatedAt
0,/credentials/9852142f-6a9e-4313-8eb9-20110fe33101,Credential,9852142f-6a9e-4313-8eb9-20110fe33101,ciec,MIGO010729SX9,valid,[],2023-01-27 01:28:54,2023-01-27 01:29:04
1,/credentials/98521378-055a-4a85-aa24-62a4a6ff6e44,Credential,98521378-055a-4a85-aa24-62a4a6ff6e44,ciec,SAMS990714DN1,valid,[],2023-01-27 01:26:54,2023-01-27 01:27:05
2,/credentials/9852134e-f056-4f6f-b081-9c8c36ee3a49,Credential,9852134e-f056-4f6f-b081-9c8c36ee3a49,ciec,MEAX800511V78,valid,[],2023-01-27 01:26:27,2023-01-27 01:26:36
3,/credentials/98520d85-26ea-4084-bf4f-bf87f3fe4f3e,Credential,98520d85-26ea-4084-bf4f-bf87f3fe4f3e,ciec,RUMC810130FP2,valid,[],2023-01-27 01:10:16,2023-01-27 01:10:25
4,/credentials/98520b90-c862-48b8-8f89-94ec32e5febb,Credential,98520b90-c862-48b8-8f89-94ec32e5febb,ciec,RIGC6106151I7,valid,[],2023-01-27 01:04:48,2023-01-27 01:04:58


In [122]:
get_all_credentials("credentials/9852142f-6a9e-4313-8eb9-20110fe33101")


{"@context":"\/contexts\/Credential","@id":"\/credentials\/9852142f-6a9e-4313-8eb9-20110fe33101","@type":"Credential","id":"9852142f-6a9e-4313-8eb9-20110fe33101","type":"ciec","rfc":"MIGO010729SX9","status":"valid","metadata":[],"createdAt":"2023-01-27 01:28:54","updatedAt":"2023-01-27 01:29:04"}


KeyError: 'hydra:member'